In [ ]:
!pip install torch torchvision

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagenet means
                         std=[0.229, 0.224, 0.225])    # Imagenet stds
])

In [14]:
dataset_path = '/content/drive/MyDrive/Dataset'
full_dataset = datasets.ImageFolder(dataset_path, transform=transform)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [22]:
model = models.resnet18(pretrained=True)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: chicken and duck

model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.6548


In [23]:
# 5. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# 6. Training Loop
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}')


Epoch 1/50, Loss: 0.3890
Epoch 2/50, Loss: 0.1987
Epoch 3/50, Loss: 0.1615
Epoch 4/50, Loss: 0.1440
Epoch 5/50, Loss: 0.1323
Epoch 6/50, Loss: 0.1210
Epoch 7/50, Loss: 0.1012
Epoch 8/50, Loss: 0.1117
Epoch 9/50, Loss: 0.1140
Epoch 10/50, Loss: 0.0953
Epoch 11/50, Loss: 0.0946
Epoch 12/50, Loss: 0.0812
Epoch 13/50, Loss: 0.1038
Epoch 14/50, Loss: 0.0927
Epoch 15/50, Loss: 0.0813
Epoch 16/50, Loss: 0.0818
Epoch 17/50, Loss: 0.0932
Epoch 18/50, Loss: 0.0795
Epoch 19/50, Loss: 0.0732
Epoch 20/50, Loss: 0.0804
Epoch 21/50, Loss: 0.0680
Epoch 22/50, Loss: 0.0739
Epoch 23/50, Loss: 0.0617
Epoch 24/50, Loss: 0.0607
Epoch 25/50, Loss: 0.0683
Epoch 26/50, Loss: 0.0648
Epoch 27/50, Loss: 0.0577
Epoch 28/50, Loss: 0.0581
Epoch 29/50, Loss: 0.0539
Epoch 30/50, Loss: 0.0622
Epoch 31/50, Loss: 0.0528
Epoch 32/50, Loss: 0.0617
Epoch 33/50, Loss: 0.0617
Epoch 34/50, Loss: 0.0534
Epoch 35/50, Loss: 0.0664
Epoch 36/50, Loss: 0.0540
Epoch 37/50, Loss: 0.0646
Epoch 38/50, Loss: 0.0441
Epoch 39/50, Loss: 0.

In [24]:
# 7. Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 8. Classification Report
print(classification_report(all_labels, all_preds, target_names=['chicken', 'duck']))

              precision    recall  f1-score   support

     chicken       0.93      0.93      0.93        96
        duck       0.97      0.97      0.97       214

    accuracy                           0.95       310
   macro avg       0.95      0.95      0.95       310
weighted avg       0.95      0.95      0.95       310

